# Welcome! #
## Enzyme and Zygote Performance Shootout ##

### Project Information ###

If you're interested in seeing a project synopsis and the benchmarks for performance we'll measuring, please take a look at our project documentation. 

### What Now? ###

Let's start by setting up Enzyme and Zygote in our environment to start testing some differentiation. Our primary benchmark is the physical timing of a differentiation operation. In order to measure this we will simply be using the `@time` Julia method. These times will then be compared to measure how fast differentiation is actually done, along with comparing other important benchmarks.

#### Import the Packages ####

In [ ]:
using Pkg

Pkg.add("Enzyme")
Pkg.add("Zygote")
Pkg.add("Plots")

In [ ]:
using Zygote
using Enzyme
using Plots

#### What Next? ####

Now that the packages have been added to the environment, we can start testing them out. First, a quick demonstration of the timing function we will be using. 

In [ ]:
function fib(n)
    if n <= 1
        return 1
    else
        return fib(n - 1) + fib(n - 2)
    end
end
    
@time fib(20)

#### Functions ####

For this shootout we will be handling differentiation for rootfinding problems. Specifically, we will be testing differentiation efficiency for aiding five different rootfinding algorithms, being Halley's, Golbabai-Javidi, Newton's, Noor's, and Zhanlav's Method.

Those methods look like such:

**Halley's Method**

$x_{n + 1} = x_{n} - \frac{2f(x_n)f^{\prime}(x_n)}{2f^{\prime}(x_n)^2 - f(x_n)f^{\prime \prime}(x_n)}$

**Golbabai-Javidi Method**

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)} - \frac{f(x_n)f^{\prime \prime}(x_n)}{2(f^{\prime \prime \prime}(x_n) - f(x_n)f^{\prime}(x_n)f^{\prime \prime}(x_n))}$

**Newton's Method**

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)}$

**Noor's Method**

$y_n = x_n - \frac{f(x_n)}{f^{\prime}(x_n)}$

$x_{n + 1} = x_{n} - \frac{f(x_n)}{f^{\prime}(x_n)} + (\frac{f(x_n)}{f^{\prime}(x_n)})\frac{f^{\prime}(y_n)}{f^{\prime}(x_n)}$

**Zhanlav's Method**

$z_n = y_n - \frac{f(y_n)}{f^{\prime}(y_n)}$

$q_n = z_n - \frac{f(z_n)}{f^{\prime}(y_n)}$

$y_{n + 1} = z_n - \frac{f(z_n) + f(q_n)}{f^{\prime}(y_n)}$

Before implementation of these various rootfinding algorithms, however, we can demonstrate some fairly basic differentiation using some basic functions below. This, similar to the time test, will present the methods we will be using and how they work.

We will be performing both of these tests using Enzyme and Zygote on three different functions, which will be defined below.

#### Test Functions ####

$f(x) = 5x^{10}$

$g(x) = sin(x)^2 + 3x^3(cos(x) - 10x)$

$h(x) = e^{\frac{5x}{2}(sin(x)^{e^x})}$

#### Zygote Method ####

Zygote is the auto differentiation tool specifically made for Julia and uses the `gradient` method for computing derivatives. Here's some implementations using our three functions.

#### Function One ####

In [ ]:
f(x) = 5x^10

In [ ]:
gf(x) = sin(x)^2 + 3x^3 * (cos(x) - 10x)

In [ ]:
hf(x) = exp((5x / 2) * (sin(x)^(exp(x))))

Below are implementations of these root finding algorithms using Enzyme and Zygote:

# Newton's with Enzyme #

In [ ]:

    # testing Newton's with Enzyme

function newton(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - fx / fpx
    end  
end

f(x) = cos(x) - x
newton(f, 1; tol=1e-15, verbose=true)

# Newton's with Zygote #

In [ ]:

    # testing Newton's with Zygote

function newton(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - fx / fpx
    end  
end

f(x) = cos(x) - x
newton(f, 1; tol=1e-15, verbose=true)

# Halley's with Enzyme #

In [ ]:

    # halley's method with Enzyme

function halley(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        fppx = first(Enzyme.autodiff(f, Active(fpx)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (2 * fx * fpx) / (2 * fpx^2 - fx * fppx)
    end  
end

f(x) = cos(x) - x
halley(f, 1; tol=1e-15, verbose=true)

# Halley's with Zygote #

In [ ]:

    # halley's method with Zygote

function halley(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        fppx = first(Zygote.gradient(f, fpx))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (2 * fx * fpx) / (2 * fpx^2 - fx * fppx)
    end  
end

f(x) = cos(x) - x
halley(f, 1; tol=1e-15, verbose=true)

# Golbabai-Javidi's with Enzyme #

In [ ]:

    # Golbabai-Javidi's method with Enzyme

function GJ(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        fppx = first(Enzyme.autodiff(f, Active(fpx)))
        fpppx = first(Enzyme.autodiff(f, Active(fppx)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) - ((fx * fppx) / (2 * (fpppx - (fx * fpx * fppx))))
    end  
end

f(x) = cos(x) - x
GJ(f, 1; tol=1e-15, verbose=true)

# Golbabai-Javidi's with Zygote #

In [ ]:

    # Golbabai-Javidi's method with Zygote

function GJ(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        fppx = first(Zygote.gradient(f, fpx))
        fpppx = first(Zygote.gradient(f, fppx))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) - ((fx * fppx) / (2 * (fpppx - (fx * fpx * fppx))))
    end  
end

f(x) = cos(x) - x
GJ(f, 1; tol=1e-15, verbose=true)

# Noor's with Enzyme #

In [ ]:

    # Noor's method with Enzyme

function noor(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        y = x - fx/fpx
        fpy = first(Enzyme.autodiff(f, Active(y)))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) + (fx / fpx) * (fpy / fpx)
    end  
end

f(x) = cos(x) - x
noor(f, 1; tol=1e-15, verbose=true)

# Noor's with Zygote #

In [ ]:

    # Noor's method with Zygote

function noor(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:100 # max number of iterations
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        y = x - fx/fpx
        fpy = first(Zygote.gradient(f, y))
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = x - (fx / fpx) + (fx / fpx) * (fpy / fpx)
    end  
end

f(x) = cos(x) - x
noor(f, 1; tol=1e-15, verbose=true)

# Zhanlav's with Enzyme #

In [ ]:

    # Zhanlav's method with Enzyme

function zhanlav(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:1000 # max number of iterations
        
        fx = f(x)
        fpx = first(Enzyme.autodiff(f, Active(x)))
        
        z = x - fx/fpx
        fz = f(z)
        fpz = first(Enzyme.autodiff(f, Active(z)))

        q = z - fz/fpz
        fq = f(q)
        
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = z - (fz + fq)/fpx
    end  
end

f(x) = cos(x) - x
zhanlav(f, 1; tol=1e-15, verbose=true)

# Zhanlav's with Zygote #

In [ ]:

    # Zhanlav's method with Zygote

function zhanlav(f, x0; tol=1e-8, verbose=false)
    x = x0
    for k in 1:1000 # max number of iterations
        
        fx = f(x)
        fpx = first(Zygote.gradient(f, x))
        
        z = x - fx/fpx
        fz = f(z)
        fpz = first(Zygote.gradient(f, z))
        
        q = z - fz/fpz
        fq = f(q)
        
        if verbose
            println("[$k] x=$x  f(x)=$fx  f'(x)=$fpx")
        end
        if abs(fx) < tol
            return x, fx, k
        end
        x = z - (fz + fq)/fpx
    end  
end

f(x) = cos(x) - x
zhanlav(f, 1; tol=1e-15, verbose=true)

# Forward difference with Enzyme & Zygote #

In [ ]:

    # testing
    # forward difference method with Enzyme and Zygote's functions.

f(x::Vector) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);
x = rand(5)
x2 = rand(5)

g = x -> Enzyme.fwddiff(f, x);
g2 = x2 -> Zygote.forwarddiff(f, x);

println(x)
println(x2)